- Spotify CSV File: data.csv
- Spotify API: xxxxx Insert API Website *****
- Need to create a git ignore .py file to store the spotify API passwords. 

In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine

Extract CSV into DataFrames

In [2]:
spotify_csv = "resources/data.csv"

Transform CSV DataFrame

In [3]:
csv_df = pd.read_csv(spotify_csv)
csv_df.head()
# Keep columns: id, name, artist, key, tempo, year, popularity

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [4]:
new_csv_df = csv_df[['id','name', 'artists','key','tempo','year','popularity']].copy()
new_csv_df.head()

# TEST: new_csv_df.info()
# There are 174389 row entries

,id,name,artists,key,tempo,year,popularity
0,0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,['Mamie Smith'],5,149.976,1920,12
1,0hbkKFIJm7Z05H8Zl9w30f,I Put A Spell On You,"[""Screamin' Jay Hawkins""]",5,86.889,1920,7
2,11m7laMUgmOKqI3oYzuhne,Golfing Papa,['Mamie Smith'],0,97.600,1920,4
3,19Lc5SfJJ5O1oaxY0fpwfh,True House Music - Xavier Santos & Carlos Gomi...,['Oscar Velazquez'],2,127.997,1920,17
4,2hJjbsLCytGsnAHfdsLejp,Xuniverxe,['Mixe'],10,122.076,1920,2


USING SPOTIPY PACKAGE
(Open source library)

In [5]:
!pip install spotipy

In [6]:
#Dependences
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import cid
from config import secret

In [7]:
# cid = 'Your Client ID'
# secret = 'Your Secret ID'
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
sp

In [8]:
for i in range (0,2):
    track_results = sp.search(q='year:2018', type='track')
    print(track_results)

artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT', 'MV', 'MW', 'MX', 'MY', 'NA', 'NE', 'NG', 'NI', 'NL', 'NO', 'NP', 'NR', 'NZ', 'OM', 'PA', 'PE', 'PG', 'PH', 'PK', 'PL', 'PS', 'PT', 'PW', 'PY', 'QA', 'RO', 'RS', 'RU', 'RW', 'SA', 'SB', 'SC', 'SE', 'SG', 'SI', 'SK', 'SL', 'SM', 'SN', 'SR', 'ST', 'SV', 'SZ', 'TD', 'TG', 'TH', 'TL', 'TN', 'TO', 'TR', 'TT', 'TV', 'TW', 'TZ', 'UA', 'UG', 'US', 'UY', 'UZ', 'VC', 'VN', 'VU', 'WS', 'XK', 'ZA', 'ZW'], 'external_urls': {'spotify': 'https://open.spotify.com/album/5gWxh24iphqQ8WDh8MBMfe'}, 'href': 'https://api.spotify.com/v1/albums/5gWxh24iphqQ8WDh8MBMfe', 'id': '5gWxh24iphqQ8WDh8MBMfe', 'images': [{'height

SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/search?q=year%3A2018&limit=50&offset=1000&type=track:
 Not found., reason: None

In [9]:
#Load dictionary into panda dataframe
pd.DataFrame.from_dict(track_results)

,tracks
href,https://api.spotify.com/v1/search?query=year%3...
items,"[{'album': {'album_type': 'album', 'artists': ..."
limit,10
next,https://api.spotify.com/v1/search?query=year%3...
offset,0
previous,None
total,258566


In [22]:
# for i in range (0,0):
#     track_results = sp.search(q='year:2020', type='artist')
#     print(track_results)

name = 'Shakira'

results = sp.search(q='artist:', type='artist')
items = results['artists']['items']
if len(items) > 0:
    artist = items[6]
    print(artist['name'], artist['images'][0]['url'])


CHAARTS Chamber Artists https://i.scdn.co/image/ab67616d0000b273b211b986534499fc28b95448


In [108]:
lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'
# spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
results = sp.artist_top_tracks(lz_uri)
for track in results['tracks'][:10]:
    print('track : ' + track['name'])
    print('audio : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

track : Stairway to Heaven - Remaster
audio : https://p.scdn.co/mp3-preview/8226164717312bc411f8635580562d67e191a754?cid=85620ab821774af180acfd6959f0d6a9
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track : Whole Lotta Love - 1990 Remaster
audio : https://p.scdn.co/mp3-preview/ce11b19a4d2de9976d7626df0717d0073863909c?cid=85620ab821774af180acfd6959f0d6a9
cover art: https://i.scdn.co/image/ab67616d0000b273fc4f17340773c6c3579fea0d

track : Immigrant Song - Remaster
audio : https://p.scdn.co/mp3-preview/8455599677a13017978dcd3f4b210937f0a16bcb?cid=85620ab821774af180acfd6959f0d6a9
cover art: https://i.scdn.co/image/ab67616d0000b27390a50cfe99a4c19ff3cbfbdb

track : Black Dog - Remaster
audio : https://p.scdn.co/mp3-preview/9b76619fd9d563a48d38cc90ca00c3008327b52e?cid=85620ab821774af180acfd6959f0d6a9
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track : Ramble On - 1990 Remaster
audio : https://p.scdn.co/mp3-preview/83383aceb01ea2

SPOTIFY API

- Objective: To extract the time signature of all tracks on spotify

In [13]:
import requests

CLIENT_ID = cid
CLIENT_SECRET = secret

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [10]:
id_list= []
for index, row in csv_df.iterrows():
    print(row['id'])
    id_list.append(row['id'])
    # print(id_list)

KFpwNBYnooFm
5zx77WvehRW4sbXWEmenY2
188vCXc4bAejLqGhpEVBGT
62bOSBdk5fEPJtPjvp7NIn
6urLjX35oXXRYtcH1b3dCh
13J9KEgQztokbVsq20sp6f
2Q85xwXagilpHKTfdrbC7M
33OAw9HSbqb9PIawSwbRSL
16iNVfWKKZPDjctIcd0KMn
7Ho9Ci4iUms5T2msr5tqoa
6xL9MJ2yt39HPPZPyGkRy8
1CXIdCgBqZg5QlUIbVrZ88
2bPYtv3okqX5zpSxtwLBuu
5DTZxyH5wlTvjwbp8Ok1U5
23xup5SPK9QGT5Rs5qUzOm
4RrSb0bSX4z3DZQdVXSOSm
6A8dnC0xkiuWN4BshmTB2I
76Eam9jEJXJrYYdJATJhET
0y8UKPyJOluqIuacosTKEv
6Fl5wnMHLvtB6pyx7W7aeq
1DA7FEKgQeyRPtU44L5DOf
0e4EqF8HiAKVS7o6bDMOf9
3JWh19WsndEFg0UFRFeTiU
7IHYGIz0dZRngGMuQciEVT
5u9cuDcYdOpSRB7fAYXXdp
3rQWfyD9N3kTkp2305bT5P
11fWR3u9wjDMW4oVDbUbyT
5whJtTCr8X0rsi414ltkei
0WSPIU3Y3bNXsGeKnzSnt1
5I0BbcHNaVODukbJbEWBkJ
3zoFkjJCH2ntsad8kfRhyi
2i4xYu1vBMvslBK2zbVvjI
3OuXuCuaZ889JdW2cX1dDw
6bwlVR9dwAWfJpXgnslbdz
1rLWsLY98u9hcWePdfN2y5
2wF80wAos6cSWuoE7JF4HG
53mrVsi49rLHIaKBiSvElG
34XoBxjnbj0z8oH1ovtp3w
5YY7ht3PCArlLjLbcTiAvh
7J2uqAiMXoYQhcxo1TkoMc
5WPXnJulBVflChZ8qIe91d
1V297gw8pk1VROba6cKpzg
5GG3knKdxKWrNboRijxeKF
2B4ZkrVA48qTzHJQ1R8eF

In [14]:
for item in id_list:

    # url = "https://api.spotify.com/v1/audio-features/{id}"

    BASE_URL = 'https://api.spotify.com/v1/'

    # Track ID from the URI
    # track_id = '6y0igZArWVi6Iz0rj35c1Y'
    track_id = item
    # actual GET request with proper header
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    r = r.json()
    # print(r)

    try:
        print(f"id: {r['id']}")
        print(f"time_signature: {r['time_signature']}")
        print('-------------------------')

    except:
        print("Error on id or time signature")

iUBq
time_signature: 4
-------------------------
id: 2gvu0Y287sZq2v62Qoo9ra
time_signature: 4
-------------------------
id: 2qCBo3PbW5dY72QngRupI0
time_signature: 4
-------------------------
id: 3PeRZ5OHm43zYP9wEMzSJD
time_signature: 4
-------------------------
id: 3fSM7e2HAOLpDWQYV4RQQJ
time_signature: 5
-------------------------
id: 57ltHEwWMgxdjHGF89laP5
time_signature: 3
-------------------------
id: 5MNdrBR0VSjNTiluJRNcdm
time_signature: 4
-------------------------
id: 5qhVYpxvr16SvMSOh2sUlE
time_signature: 4
-------------------------
id: 60b8VQg6oDrzPsmTkQ3Zsx
time_signature: 3
-------------------------
id: 61mLhlaN1WMJNJv4zvJGe2
time_signature: 3
-------------------------
id: 67RhK21dHNYTvPzBhI96Mx
time_signature: 4
-------------------------
id: 6957wQq4CP6YqJZQJJkQth
time_signature: 4
-------------------------
id: 6Drgn07ePwaj6sFs8UGfL8
time_signature: 4
-------------------------
id: 6NrXYND5iddO5OW80drYBZ
time_signature: 4
-------------------------
id: 6PGne8ylB0ULe6TH33vbOF
t

KeyboardInterrupt: 